In [1]:
import numpy as np
import pandas as pd
import time

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression

In [2]:
# cost or loss function
def cost(Y, Yhat):
    return np.mean((Yhat - Y) ** 2)

In [3]:
path = '../../data/KNN_Linear_Regression/SAT_GPA.csv'

df = pd.read_csv(path)
df.head()

,SAT,GPA
0,1714,2.40
1,1664,2.52
2,1760,2.54
3,1685,2.74
4,1693,2.83


In [4]:
X = df["SAT"]
y = df["GPA"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=30/84, random_state=1)

In [5]:
X_train = X_train.to_numpy().reshape(-1, 1)
X_test = X_test.to_numpy().reshape(-1, 1)

y_train = y_train.to_numpy().reshape(-1, 1)
y_test = y_test.to_numpy().reshape(-1, 1)

**Huấn luyện mô hình trên tập Train**

In [6]:
d0 = 1
d1 = h = 100 # size of hidden layer
d2 = C = 1

# initialize parameters randomly
W1 = 0.01*np.random.randn(d0, d1)
b1 = np.zeros((d1, 1))
W2 = 0.01*np.random.randn(d1, d2)
b2 = np.zeros((d2, 1))

N = X_train.T.shape[1]
eta = 0.0005 # learning rate

start_time = time.time()

for i in range(10000):
    ## Feedforward
    Z1 = np.dot(W1.T, X_train.T) + b1
    A1 = np.where(Z1 > 0, Z1, 0.01 * Z1)  # LeakyReLU
    Z2 = np.dot(W2.T, A1) + b2
    Yhat = Z2

    # print loss after each 1000 iterations
    if i %1000 == 0:
        # compute the loss: average cross-entropy loss
        loss = cost(y_train, Yhat)
        print("iter %d, loss: %f" %(i, loss))

    # backpropagation
    E2 = (Yhat - y_train.T)/N
    dW2 = np.dot(A1, E2.T)
    db2 = np.sum(E2, axis = 1, keepdims = True)
    E1 = np.dot(W2, E2)
    E1 = np.where(Z1 > 0, E1, 0.01 * E1)  # Gradient of LeakyReLU
    dW1 = np.dot(X_train.T, E1.T)
    db1 = np.sum(E1, axis = 1, keepdims = True)
    
    # Gradient clipping (To avoid booming gradient)
    clip_value = 1.0
    dW1 = np.clip(dW1, -clip_value, clip_value)
    dW2 = np.clip(dW2, -clip_value, clip_value)
    db1 = np.clip(db1, -clip_value, clip_value)
    db2 = np.clip(db2, -clip_value, clip_value)
    
    # Gradient Descent update
    W1 += -eta*dW1
    b1 += -eta*db1
    W2 += -eta*dW2
    b2 += -eta*db2

training_time = time.time() - start_time
print(f"\nTraining Time: %f s" % training_time)

iter 0, loss: 0.467664
iter 1000, loss: 0.343570
iter 2000, loss: 0.334343
iter 3000, loss: 0.332861
iter 4000, loss: 0.332679
iter 5000, loss: 0.332593
iter 6000, loss: 0.332596
iter 7000, loss: 0.332574
iter 8000, loss: 0.332681
iter 9000, loss: 0.332575

Training Time: 0.492591 s


**Dự đoán đầu ra trên tập Train**

In [7]:
Z1 = np.dot(W1.T, X_train.T) + b1
A1 = np.maximum(Z1, 0)
Z2 = np.dot(W2.T, A1) + b2

Z2

array([[3.00863641, 2.91275828, 2.69403629, 2.95620306, 2.90077351,
        3.07455262, 2.75995251, 2.59666007, 2.91425637, 2.54123052,
        2.79590681, 2.94272019, 2.74946584, 2.96219544, 2.8318611 ,
        2.9741802 , 2.70152677, 2.52475147, 2.5846753 , 2.91725256,
        2.68504772, 2.65958009, 2.7284925 , 2.95919925, 2.58167911,
        2.50827241, 2.67605914, 2.66707057, 2.84384587, 2.98167068,
        2.78841633, 2.85283445, 2.54422671, 2.55321529, 3.04459071,
        2.67306295, 2.70452296, 3.04309261, 2.67605914, 2.77643156,
        2.6550858 , 2.71650773, 2.75246203, 2.65958009, 2.68055343,
        2.50977051, 2.64609723, 2.85283445, 2.51875908, 3.03560213,
        2.78841633, 2.9756783 , 2.61613531, 2.94421829]])

**Thời gian predict trung bình**

In [8]:
num_runs = 10
prediction_times = []

for _ in range(num_runs):
    start_time = time.time()
    
    Z1 = np.dot(W1.T, X_train.T) + b1
    A1 = np.maximum(Z1, 0)
    Z2 = np.dot(W2.T, A1) + b2
    
    prediction_time = time.time() - start_time
    prediction_times.append(prediction_time)

avg_prediction_time = np.mean(prediction_times)
print("Average Predict Time: %f s" % avg_prediction_time)

Average Predict Time: 0.000000 s


**Đánh giá độ chính xác:**

* R square = -2.948 < 0 => Mô hình không phù hợp 

* MSE = 0.313 là khá cao khi xét trong khoảng giá trị dự đoán

In [9]:
print("R square:", r2_score(y_train, Z2[0]))
print("MSE:", mean_squared_error(y_train, Z2[0]))

R square: -2.94834974793418
MSE: 0.3136479132206353


**Huấn luyện mô hình trên tập Test**

In [10]:
d0 = 1
d1 = h = 100 # size of hidden layer
d2 = C = 1
# initialize parameters randomly
W1 = 0.01*np.random.randn(d0, d1)
b1 = np.zeros((d1, 1))
W2 = 0.01*np.random.randn(d1, d2)
b2 = np.zeros((d2, 1))

N = X_test.T.shape[1]
eta = 0.0005 # learning rate

start_time = time.time()

for i in range(10000):
    ## Feedforward
    Z1 = np.dot(W1.T, X_test.T) + b1
    A1 = np.where(Z1 > 0, Z1, 0.01 * Z1)  # LeakyReLU
    Z2 = np.dot(W2.T, A1) + b2
    Yhat = Z2

    # print loss after each 1000 iterations
    if i %1000 == 0:
        # compute the loss: average cross-entropy loss
        loss = cost(y_test, Yhat)
        print("iter %d, loss: %f" %(i, loss))

    # backpropagation
    E2 = (Yhat - y_test.T)/N
    dW2 = np.dot(A1, E2.T)
    db2 = np.sum(E2, axis = 1, keepdims = True)
    E1 = np.dot(W2, E2)
    E1 = np.where(Z1 > 0, E1, 0.01 * E1)  # Gradient of LeakyReLU
    dW1 = np.dot(X_test.T, E1.T)
    db1 = np.sum(E1, axis = 1, keepdims = True)
    
    # Gradient clipping
    clip_value = 1.0
    dW1 = np.clip(dW1, -clip_value, clip_value)
    dW2 = np.clip(dW2, -clip_value, clip_value)
    db1 = np.clip(db1, -clip_value, clip_value)
    db2 = np.clip(db2, -clip_value, clip_value)
    
    # Gradient Descent update
    W1 += -eta*dW1
    b1 += -eta*db1
    W2 += -eta*dW2
    b2 += -eta*db2

training_time = time.time() - start_time
print(f"\nTraining Time: %f s" % training_time)

iter 0, loss: 9.589682
iter 1000, loss: 0.153308
iter 2000, loss: 0.150617
iter 3000, loss: 0.142201
iter 4000, loss: 0.141465
iter 5000, loss: 0.141406
iter 6000, loss: 0.141329
iter 7000, loss: 0.141333
iter 8000, loss: 0.141359
iter 9000, loss: 0.141325

Training Time: 0.450134 s


**Dự đoán đầu ra trên tập Test**

In [11]:
Z1 = np.dot(W1.T, X_test.T) + b1
A1 = np.maximum(Z1, 0)
Z2 = np.dot(W2.T, A1) + b2

Z2

array([[3.46875201, 3.8677027 , 3.78149728, 3.70932529, 4.10025688,
        4.04011356, 3.75944472, 3.55295266, 3.9118078 , 3.45872812,
        3.68526796, 3.72135395, 3.61510075, 3.6993014 , 3.26626949,
        3.66321541, 3.8677027 , 3.61510075, 3.64116286, 3.40259435,
        3.72937306, 3.51887144, 3.9739559 , 3.69729663, 3.71333484,
        3.74340651, 3.86168837, 3.81357371, 3.57099565, 3.97997023]])

**Thời gian predict trung bình**

In [12]:
num_runs = 10
prediction_times = []

for _ in range(num_runs):
    start_time = time.time()
    
    Z1 = np.dot(W1.T, X_test.T) + b1
    A1 = np.maximum(Z1, 0)
    Z2 = np.dot(W2.T, A1) + b2
    
    prediction_time = time.time() - start_time
    prediction_times.append(prediction_time)

avg_prediction_time = np.mean(prediction_times)
print("Average Predict Time: %f s" % avg_prediction_time)

Average Predict Time: 0.000000 s


**Đánh giá độ chính xác:**

* R square = -1.746 < 0 => Mô hình không phù hợp 

* MSE = 0.148 là hơi cao khi xét trong khoảng giá trị dự đoán

In [13]:
print("R square:", r2_score(y_test, Z2[0]))
print("MSE:", mean_squared_error(y_test, Z2[0]))

R square: -1.7469342194889128
MSE: 0.14828683432593015


**Sử dụng mô hình hồi quy tuyến tính**

In [14]:
linR = LinearRegression()

start_time = time.time()

linR.fit(X_train, y_train)

training_time = time.time() - start_time
print("Training Time: %f s" % training_time)


num_runs = 10
prediction_times = []

for _ in range(num_runs):
    start_time = time.time()
    y_pred = linR.predict(X_test)
    prediction_time = time.time() - start_time
    prediction_times.append(prediction_time)
    
avg_prediction_time = np.mean(prediction_times)
print("Average Predict Time: %f s" % avg_prediction_time)

print("R square:", r2_score(y_test, y_pred))
print("MSE:", mean_squared_error(y_test, y_pred))

Training Time: 0.002434 s
Average Predict Time: 0.000037 s
R square: 0.05950572997060655
MSE: 0.05077038868090673


**So sánh với mô hình ANN:**

* Thời gian training: Mô hình `Linear Regression` nhanh hơn nhiều so với mô hình `ANN`

* Thời gian predict: Hai mô hình tương đương nhau

* R square: Mô hình `Linear Regression` có R square = 0.059 nằm trong khoảng (0, 1), cho thấy các biến độc lập giải thích được khoảng 5.9% sự biến thiên của biến phụ thuộc. nhưng như thế vẫn là tốt hơn so với `ANN` khi R square < 0 

* MSE: Sai số trung bình của mô hình `Linear Regression` là 0.051, là nhỏ hơn hẳn so với mô hình `ANN`, cho thấy `Linear Regression` dự đoán chính xác hơn

**Mô hình `ANN` với số chiều layer ẩn là 75**

In [15]:
d0 = 1
d1 = h = 75 # size of hidden layer
d2 = C = 1
# initialize parameters randomly
W1 = 0.01*np.random.randn(d0, d1)
b1 = np.zeros((d1, 1))
W2 = 0.01*np.random.randn(d1, d2)
b2 = np.zeros((d2, 1))

N = X_train.T.shape[1]
eta = 0.0005 # learning rate

start_time = time.time()

for i in range(10000):
    ## Feedforward
    Z1 = np.dot(W1.T, X_train.T) + b1
    A1 = np.where(Z1 > 0, Z1, 0.01 * Z1)  # LeakyReLU
    Z2 = np.dot(W2.T, A1) + b2
    Yhat = Z2

    # print loss after each 1000 iterations
    if i %1000 == 0:
        # compute the loss: average cross-entropy loss
        loss = cost(y_train, Yhat)
        print("iter %d, loss: %f" %(i, loss))

    # backpropagation
    E2 = (Yhat - y_train.T)/N
    dW2 = np.dot(A1, E2.T)
    db2 = np.sum(E2, axis = 1, keepdims = True)
    E1 = np.dot(W2, E2)
    E1 = np.where(Z1 > 0, E1, 0.01 * E1)  # Gradient of LeakyReLU
    dW1 = np.dot(X_train.T, E1.T)
    db1 = np.sum(E1, axis = 1, keepdims = True)
    
    # Gradient clipping
    clip_value = 1.0
    dW1 = np.clip(dW1, -clip_value, clip_value)
    dW2 = np.clip(dW2, -clip_value, clip_value)
    db1 = np.clip(db1, -clip_value, clip_value)
    db2 = np.clip(db2, -clip_value, clip_value)
    
    # Gradient Descent update
    W1 += -eta*dW1
    b1 += -eta*db1
    W2 += -eta*dW2
    b2 += -eta*db2

training_time = time.time() - start_time
print(f"\nTraining Time: %f s" % training_time)


Z1 = np.dot(W1.T, X_test.T) + b1
A1 = np.maximum(Z1, 0)
Z2 = np.dot(W2.T, A1) + b2

print("R square:", r2_score(y_test, Z2[0]))
print("MSE:", mean_squared_error(y_test, Z2[0]))

iter 0, loss: 21.215327
iter 1000, loss: 0.171394
iter 2000, loss: 0.170215
iter 3000, loss: 0.178774
iter 4000, loss: 0.180008
iter 5000, loss: 0.180206
iter 6000, loss: 0.180227
iter 7000, loss: 0.180207
iter 8000, loss: 0.180231
iter 9000, loss: 0.180235

Training Time: 0.458576 s
R square: -1.2751496073768074
MSE: 0.12281864287181972


Khi giảm số chiều layer ẩn xuống 75.

Ta thấy `R square` của mô hình tăng lên và `MSE` giảm xuống.

=> Mô hình trở nên tốt hơn

**Mô hình `ANN` với số chiều layer ẩn là 50**

In [16]:
d0 = 1
d1 = h = 50 # size of hidden layer
d2 = C = 1
# initialize parameters randomly
W1 = 0.01*np.random.randn(d0, d1)
b1 = np.zeros((d1, 1))
W2 = 0.01*np.random.randn(d1, d2)
b2 = np.zeros((d2, 1))

N = X_train.T.shape[1]
eta = 0.0005 # learning rate

start_time = time.time()

for i in range(10000):
    ## Feedforward
    Z1 = np.dot(W1.T, X_train.T) + b1
    A1 = np.where(Z1 > 0, Z1, 0.01 * Z1)  # LeakyReLU
    Z2 = np.dot(W2.T, A1) + b2
    Yhat = Z2

    # print loss after each 1000 iterations
    if i %1000 == 0:
        # compute the loss: average cross-entropy loss
        loss = cost(y_train, Yhat)
        print("iter %d, loss: %f" %(i, loss))

    # backpropagation
    E2 = (Yhat - y_train.T)/N
    dW2 = np.dot(A1, E2.T)
    db2 = np.sum(E2, axis = 1, keepdims = True)
    E1 = np.dot(W2, E2)
    E1 = np.where(Z1 > 0, E1, 0.01 * E1)  # Gradient of LeakyReLU
    dW1 = np.dot(X_train.T, E1.T)
    db1 = np.sum(E1, axis = 1, keepdims = True)
    
    # Gradient clipping
    clip_value = 1.0
    dW1 = np.clip(dW1, -clip_value, clip_value)
    dW2 = np.clip(dW2, -clip_value, clip_value)
    db1 = np.clip(db1, -clip_value, clip_value)
    db2 = np.clip(db2, -clip_value, clip_value)
    
    # Gradient Descent update
    W1 += -eta*dW1
    b1 += -eta*db1
    W2 += -eta*dW2
    b2 += -eta*db2

training_time = time.time() - start_time
print(f"\nTraining Time: %f s" % training_time)

Z1 = np.dot(W1.T, X_test.T) + b1
A1 = np.maximum(Z1, 0)
Z2 = np.dot(W2.T, A1) + b2

print("R square:", r2_score(y_test, Z2[0]))
print("MSE:", mean_squared_error(y_test, Z2[0]))

iter 0, loss: 11.499286
iter 1000, loss: 0.147084
iter 2000, loss: 0.145789
iter 3000, loss: 0.143994
iter 4000, loss: 0.142153
iter 5000, loss: 0.139940
iter 6000, loss: 0.137201
iter 7000, loss: 0.136895
iter 8000, loss: 0.135084
iter 9000, loss: 0.135063

Training Time: 0.399914 s
R square: -3.6264598199223403
MSE: 0.24974863830559435


Khi giảm số chiều xuống 50.

Ta thấy `R square` lại giảm đi rất nhiều và `MSE` lại tăng lên.

=> Đã giảm số chiều layer ẩn quá nhiều khiến mô hình không học được đặc trưng và dẫn đến underfitting.

**Nhận xét mối liên hệ giữa siêu tham số số chiều layer ẩn:**

* Số chiều layer ẩn quyết định dung lượng mô hình. 

* Số chiều lớn tăng khả năng học nhưng dễ overfitting; Số chiều nhỏ giảm nguy cơ overfitting nhưng lại tăng nguy cơ underfitting.